In [ ]:
#@title AUTOENCODER
import numpy as np
from keras.layers import Input, Conv1D, MaxPooling1D, UpSampling1D, Dropout,Cropping1D
from keras.models import Model
from keras.optimizers import Adam
from sklearn.metrics import accuracy_score
from keras import regularizers
from keras.backend import clear_session


# Создаем модель автоэнкодера
def create_autoencoder(input_shape):
    input_data = Input(shape=input_shape)
    print(input_data.shape)

    x = Conv1D(32, 3, activation='gelu', padding='causal')(input_data)
    x = MaxPooling1D(2, padding='same')(x)
    # x = Dropout(0.1)(x)  # Add dropout layer after pooling
    x = Conv1D(16, 3, activation='gelu', padding='causal')(x)
    encoded = MaxPooling1D(2, padding='same', name='encoded')(x)

    x = Conv1D(16, 3, activation='gelu', padding='causal')(encoded)
    x = UpSampling1D(2)(x)
    # x = Dropout(0.1)(x)  # Add dropout layer after upsampling
    x = Conv1D(32, 3, activation='gelu', padding='causal')(x)
    x = UpSampling1D(2)(x)
    decoded = Conv1D(input_shape[1], 3, activation='sigmoid', padding='causal')(x)
    # decoded = Cropping1D(cropping=(1, 0))(decoded)  # Add cropping layer to match the input shape

    print(decoded.shape)

    autoencoder = Model(input_data, decoded)
    autoencoder.compile(optimizer=Adam(1e-4), loss='mse')
    return autoencoder

# Загрузка данных
train_data = X ## isnert your data

# Создаем и обучаем автоэнкодер
input_shape = (train_data.shape[1], train_data.shape[2]) # modify for yor shape
autoencoder = create_autoencoder(input_shape)

clear_session()
autoencoder.fit(train_data, train_data, epochs=500, verbose=1)


In [ ]:
# autoencoder.save('autoencoder_drop2.h5')

In [ ]:
#@title AUTOENCODER.INFERENCE
from execthatcell import *
from keras.models import load_model
from sklearn.metrics import accuracy_score
import numpy as np
from keras.models import Model
from scipy.spatial.distance import cosine,euclidean
from sklearn.metrics import mean_squared_error

# autoencoder = load_model('autoencoder_drop.h5')

FIRST_DROP = 382
TEST_MODE = True

while FIRST_DROP != 290:

    FIRST_DROP -= 1
    execthatcell("load_csv", cell_marker="#@#", latest=True)


    K = WND
    offst = K*15
    x_input = np.transpose(np.vstack((
        test['body_size'].to_numpy(),
        # test['range_size'].to_numpy(),
        # test['upper_shadow_size'].to_numpy(),
        # test['lower_shadow_size'].to_numpy(),

        # nothing_df['body_size'][offst:offst+K].to_numpy(),
        # nothing_df['range_size'][offst:offst+K].to_numpy(),
        # nothing_df['upper_shadow_size'][offst:offst+K].to_numpy(),
        # nothing_df['lower_shadow_size'][offst:offst+K].to_numpy(),
        # # nothing_df buy_df
        )))

    train_data = X
    test_data = x_input.reshape((1, x_input.shape[0], x_input.shape[1]))

    # Извлекаем признаки из тестового набора
    encoder = Model(autoencoder.input, autoencoder.get_layer('encoded').output)
    test_features = encoder.predict(test_data,verbose=0)

    # Сравниваем с тестовым набором через евклидово расстояние
    train_features = encoder.predict(train_data,verbose=0)
    distances = [euclidean(train_features[i].flatten(), test_features.flatten()) for i in range(len(train_data))] #np.linalg.norm(train_features - test_features, axis=(1, 2))

    # Выбираем ближайший временной ряд
    closest_index = np.argmin(distances)
    closest_series = train_data[closest_index]

    # # Вычисляем процент точности
    # accuracy = accuracy_score(np.round(test_data).flatten(), np.round(closest_series).flatten()) * 100
    # # if accuracy>=60:
    # print(f"Точность: {accuracy:.2f}%\n")



    # # Вычисление RMSE
    # rmse = mean_squared_error(test_data.flatten(), closest_series.flatten(), squared=False)
    # # rmse = np.sqrt(np.mean((test_data - closest_series)**2))

    # # Вывод результатов
    # print(f"RMSE: {rmse:.2f}")

    plt.plot((test_data.reshape(-1,1)).flatten())
    plt.plot((closest_series.reshape(-1,1)).flatten())
    plt.title(FIRST_DROP)
    plt.show()
    plt.close()
    print('\n')
